In [1]:
import pandas as pd
import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objs as go
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel("DLR scenario data-WORLD.xlsx", sheet_name = "Final energy demand 1.5C")
data = df.iloc[89:117,2:12]
data = data.drop(columns = ['Unnamed: 3'], axis = 1).reset_index(drop = True)
data = data.drop(data.index[5]).drop(data.index[16]).drop(data.index[23]).\
        drop(data.index[0]).drop(data.index[6]).drop(data.index[17]).\
        drop(data.index[24]).drop(data.index[1]).reset_index(drop = True)

In [3]:
data = data.rename(columns={'Unnamed: 2':'Carriers',
                     'Unnamed: 4':'2015',
                     'Unnamed: 5':'2020',
                     'Unnamed: 6':'2025',
                     'Unnamed: 7':'2030',
                     'Unnamed: 8':'2035', 
                     'Unnamed: 9':'2040', 
                     'Unnamed: 10':'2045', 
                     'Unnamed: 11':'2050'})

In [4]:
data['Carriers'] = data['Carriers'].map(lambda x: x.lstrip('  - '))
data = data.set_index(['Carriers'], drop = True)

In [5]:
FFs = ['Coal & Lignite', 'Gas', 'Oil & Diesel']
Power = ['Nuclear', 'Hydrogen Power',
       'Hydro Power', 'Wind', 'PV', 'Biomass Power', 'Geothermal Power',
       'Solar thermal power plants', 'Ocean energy']
OtherHeat = ['Solar Heat', 'Biomass',
       'Geothermal', 'H Process', 'RE district heat', 'Biofuels ', 'Synfuels',
       'H Fuel']
for i in data.index:
    if i in FFs:
        data.loc[i,'Carrier_Type'] = "FFs"
    elif i in Power:
        data.loc[i,'Carrier_Type'] = "Power"
    else:
        data.loc[i,'Carrier_Type'] = "Other Heat"

In [6]:
year = []
total_energy_use = []
carrier = []
carrier_type = []


for i in range(data.shape[0]): #row
    for j in range(data.shape[1]-1): #column
        total_energy_use.append(data.iloc[i,j]) 
        carrier.append(data.index[i]) 
        year.append(data.columns[j]) 
        carrier_type.append(data.iloc[i,8])
          

data_cleaned = pd.DataFrame(data = year, columns = ["Year"])
data_cleaned["Carrier Type"] = carrier_type
data_cleaned["Carrier"] = carrier
data_cleaned["Total Energy Use"] = total_energy_use

In [7]:
data_cleaned['Percent'] = data_cleaned['Total Energy Use'].map(lambda x: x/sum(data_cleaned['Total Energy Use']))

In [8]:
sequence = ['PV','Solar thermal power plants','Biomass Power','Wind','Ocean energy', 
            'Hydrogen Power','Hydro Power', 'Geothermal Power','Nuclear','Gas',
            'Coal & Lignite', 'Oil & Diesel','Solar Heat','Biomass','Geothermal',
           'H Process', 'RE district heat', 'H Fuel', 'Synfuels','Biofuels ' ]

data_sequence = []
for i in range(len(data_cleaned['Carrier'])):
    if data_cleaned['Carrier'][i] in sequence:
        data_sequence.append(sequence.index(data_cleaned['Carrier'][i]))
data_cleaned["Sequence"]=data_sequence
data_cleaned = data_cleaned.sort_values(by="Sequence")

In [9]:
cleaned_Final_Energy_Demand = data_cleaned.to_csv(r'C:\Users\wangr\Desktop\cleaned_FINAL_Energy_Demand_1.5C.csv', index = False)

In [10]:
a=data_cleaned.groupby(['Carrier','Year']).agg({'Total Energy Use':'sum', 'Percent':'sum'})
a.reset_index()
a=a.reset_index(level='Year')

In [11]:
sources=data_cleaned['Carrier'].unique()

In [12]:
colors={'PV':'rgb(251,173,60)','Solar thermal power plants':'rgb(236,112,20)','Biomass Power':'rgb(113,50,141)',
        'Wind':'rgb(68,130,155)','Ocean energy':'rgb(2,56,88)','Hydro Power':'rgb(15,71,153)',
        'Geothermal Power':'rgb(0,109,44)','Hydrogen Power':'rgb(255,0,0)','Nuclear':'rgb(217,217,217)',
        'Gas':'rgb(150,150,150)','Coal & Lignite':'rgb(82,82,82)','Oil & Diesel':'rgb(0,0,0)',
        'Solar Heat':'rgb(255,111,0)','Biomass':'rgb(0,109,44)','Geothermal':'rgb(113,50,141)',
        'H Process':'rgb(255,0,0)','RE district heat':'rgb(166,54,3)','H Fuel':'rgb(223,101,176)',
       'Synfuels': 'rgb(68,130,155)','Biofuels ':'rgb(127,205,187)'}

In [13]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([
        dcc.Graph(id = "graph2")
        ]),
    html.Div([
        html.Label("Graphs"), 
        dcc.Dropdown(
            id = "w_dropdown",
            options=[
                         {'label': 'Global Expansion of Renewables to Achieve the 1.5C Goal', 'value': 'data'},
                ],
            value = "Total Energy Use",
            multi = False,
            clearable = False,
            style = {"width": "50%"}
        )
        ])
])



@app.callback(
    Output(component_id = "graph2", component_property = 'figure'),
    [Input(component_id = "w_dropdown", component_property = 'value')]
)

def update_graph(my_dropdown):
    import plotly.graph_objs as go
    fig = go.Figure()
    for source in sources:   
    
        data = a.loc[source,{'Year','Total Energy Use','Percent'}].reset_index()
        fig.add_trace(go.Scatter(x = a.loc[source,'Year'],
                        y =a.loc[source,'Total Energy Use'],
                                 mode="none",
                        fillcolor=colors[source],
                                text=round(a.loc[source,'Percent']*100,2),
                                         hovertemplate =
                                        '<br><b>'+source+'</b></b>:<extra></extra>'+
                                        '<br><b>Electricity</b></b>'+
                                        '<br><b>Year</b>: %{x}'+
                                        '<br><b>Demand</b>: %{y}'+
                                        '<br><b>Electrification</b>: %{text}%<br>',        
                        name = source,
                        stackgroup='one'))

    fig.add_shape(type="line",
                x0=2020, y0=0, x1=2020, y1=330000,
                line=dict(color='lightgrey',width=0.3,dash="dash"))

    fig.add_shape(type="line",
                x0=2030, y0=0, x1=2030, y1=300000,
                line=dict(color='lightgrey',width=0.3,dash="dash"))

    fig.add_shape(type="line",
                x0=2040, y0=0, x1=2040, y1=260000,
                line=dict(color='lightgrey',width=0.3,dash="dash"))

    fig.add_trace(go.Scatter(
    x=[2015.5,2015.5,2015.5,2015.5],
    y=[21000, 90000, 160000, 285000],
    mode="text",
    text=["Nuclear", 'Gas','Coal','Oil'],
    textposition="top right",
    textfont=dict(
        family="Arial",
        size=12,
        color="white")))

    fig.add_trace(go.Scatter(
    x=[2049.5,2049.5,2049.5,2049.5,2049.5,2049.5,2049.5,2049.5, 2049.5,2049.5],
    y=[4000, 50000, 70000, 123500, 150000, 172000,198000,211000, 222000, 240000],
    mode="text",
    text=["Solar PV", 'Solar CSP','Wind','Hydro','Solar Heat', 'Biomass', 'Geothermal','Hydrogen', 'RE District','RE Fuels'],
    textposition="top left",
    textfont=dict(
        family="Arial",
        size=12,
        color="white")))

    fig.update_yaxes(ticktext=['50,000','100,000','150,000','200,000','250,000','300,000','350,000'],
                tickvals=[50000,100000,150000,200000,250000,300000,350000])

    fig.update_layout(title = 'Global Expansion of Renewables to Achieve the 1.5C Goal',
                      font=dict(family="Arial", size=15, color="black"),
                 xaxis = dict(showgrid=False),
                 yaxis = dict(title = 'Final Energy Demand (PJ)',showgrid=False),
                 width=1000,
                 height=800,
                 plot_bgcolor='white',
                showlegend=False
                     )

    fig.show()


    return fig


if __name__ == '__main__':
    app.run_server(debug=True, mode='inline',host="127.0.0.1", port=8090)